# Activity coeficient code

In [7]:
from numpy import *
import numpy as np
import random
import re
import sys
from matplotlib import pyplot as plt
import os

In [8]:
profiles=[[],[]]

def sacgenerator(fileName0,fileName1):
    a=int(0)
    for fileName in (fileName0,fileName1):
        
        f = open(fileName)
        #read through each line in the file
        lines = f.readlines()
        lenth = len(lines)
        data_lines = f.readlines(69)
        # search cosmo 'area' and 'volume' in the file then convert unit to A
        f.close

        for i in range(lenth):
            line_data = re.findall(r"\-?\d+\.?\d*",lines[i])
            profiles[a].append(double(line_data[1]))
            # make 7 lists containing data
            print('*',end='')
        print('The file',fileName,'is processed.')
        a=a+1
    return "^_^"

In [9]:
sacgenerator('BM.txt','CO.txt')


FileNotFoundError: [Errno 2] No such file or directory: 'BM.txt'

In [10]:
%%timeit -r1 -n1
# define some constants
EO = float64(2.395*10**-4)
AEFFPRIME = float64(7.5)
RGAS = float64(0.001987)
VNORM = float64(66.69)
ANORM = float64(79.53)
COMPSEG = 51 #NUMBER OF INTERVALS FOR THE SIGMA PROFILE
EPS = float64(3.667) #(LIN AND SANDLER USE A CONSTANT FPOL WHICH YEILDS EPS=3.68)
COORD = float64(10.0) #(KLAMT USED 7.2)  
SIGMAHB = float64(0.0084)
CHB = float64(85580.0)
COMP =2 
FPOL = (EPS-1.0)/(EPS+0.5)
ALPHA = (0.3*AEFFPRIME**(1.5))/(EO)
ALPHAPRIME = FPOL*ALPHA
SYSTEMP = 300#K
VNORM = float64(66.69)
ANORM = float64(79.53)
# define some lists
numer = np.zeros(COMPSEG,dtype='float64')
demon = np.zeros(COMPSEG,dtype='float64')
profile = np.zeros(COMPSEG,dtype='float64')
CONVERG = np.zeros(COMPSEG,dtype='float64')
DELTAW = np.zeros((COMPSEG,COMPSEG),dtype='float64')
SEGGAMMA = np.ones(COMPSEG)
SEGGAMMAOLD = np.ones(COMPSEG,dtype='float64')
CONPR =  np.ones((2,COMPSEG),dtype='float64')
chgden = np.linspace(-0.025,0.025,51)

cot=0
for o in range(1):
    cot+=1
    print('*',end='')
    frac1 = o/500
    frac2 = 1-frac1
    
# calculate the sigma profile of binary mixture.
    for i in range(COMPSEG):
        
        numer[i]=frac1*profiles[0][i]+frac2*profiles[1][i]
        demon[i]=frac1*sum(profiles[0])+frac2*sum(profiles[1])
        
    profile = numer/demon
    
# calculate the delta W
    for i in range(COMPSEG):
        for j in range(COMPSEG):
            if chgden[i]>=chgden[j]:
                sigma_acc = chgden[i]
                sigma_don = chgden[j]
            else:
                sigma_acc = chgden[j]
                sigma_don = chgden[i]

            DELTAW[i,j] = (ALPHAPRIME/2)*(chgden[i]+chgden[j])**2 + CHB*max(0,(sigma_acc - SIGMAHB))*min(0,(sigma_don + SIGMAHB))
    
# segment acitivity coefficient for binary mixture.
    SEGGAMMA = np.ones(COMPSEG)
    
    
    while True:
        SEGGAMMAOLD = copy(SEGGAMMA)
        for i in range(COMPSEG):

            SUMMATION=0


            for j in range(COMPSEG):


                SUMMATION = SUMMATION + profile[j]* SEGGAMMAOLD[j] * np.exp(-DELTAW[i,j]/(RGAS*SYSTEMP))

            SEGGAMMA[i]=np.exp(-np.log(SUMMATION))
            SEGGAMMA[i]=(SEGGAMMA[i]+SEGGAMMAOLD[i])/2
        for i in range(COMPSEG):
            CONVERG[i]=abs((SEGGAMMA[i]-SEGGAMMAOLD[i])/SEGGAMMAOLD[i])
        if max(CONVERG)<=0.0000001:
            break
            
    #plt.plot(chgden,log(SEGGAMMA))
# segment acitivity coefficient for pure components.       


    SEGGAMMAPR1 = np.ones(COMPSEG,dtype='float64')

    SEGGAMMAOLDPR1= np.ones(COMPSEG,dtype='float64')

    acosmo1=float64(sum(profiles[0]))

    while True:
        SEGGAMMAOLDPR1 = copy(SEGGAMMAPR1)
        for i in range(COMPSEG):

            SUMMATION=0


            for j in range(COMPSEG):


                SUMMATION = SUMMATION + profiles[0][j]/acosmo1* SEGGAMMAOLDPR1[j] * np.exp(-DELTAW[i,j]/(RGAS*SYSTEMP))

            SEGGAMMAPR1[i]=np.exp(-np.log(SUMMATION))
            SEGGAMMAPR1[i]=(SEGGAMMAPR1[i]+SEGGAMMAOLDPR1[i])/2

        for i in range(COMPSEG):
            CONVERG[i]=abs((SEGGAMMAPR1[i]-SEGGAMMAOLDPR1[i])/SEGGAMMAOLDPR1[i])
        if max(CONVERG)<=0.0000001:
            break



    SEGGAMMAPR2 = np.ones(COMPSEG,dtype='float64')

    SEGGAMMAOLDPR2 = np.ones(COMPSEG,dtype='float64')

    acosmo2=float64(sum(profiles[1]))

    while True:
        SEGGAMMAOLDPR2 = copy(SEGGAMMAPR2)
        for i in range(COMPSEG):

            SUMMATION=0


            for j in range(COMPSEG):


                SUMMATION = SUMMATION + profiles[1][j]/acosmo2* SEGGAMMAOLDPR2[j] * np.exp(-DELTAW[i,j]/(RGAS*SYSTEMP))

            SEGGAMMAPR2[i]=np.exp(-np.log(SUMMATION))
            SEGGAMMAPR2[i]=(SEGGAMMAPR2[i]+SEGGAMMAOLDPR2[i])/2

        for i in range(COMPSEG):
            CONVERG[i]=abs((SEGGAMMAPR2[i]-SEGGAMMAOLDPR2[i])/SEGGAMMAOLDPR2[i])
        if max(CONVERG)<=0.0000001:
            break




#>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>        


    vcosmo=np.array([564,132],dtype='float64')

    RNORM = vcosmo/VNORM
    QNORM = [acosmo1/ANORM,acosmo2/ANORM]

    BOTTHETA = frac1*QNORM[0] + frac2*QNORM[1]
    BOTPHI = frac1*RNORM[0] + frac2*RNORM[1]

    THETA1 = frac1*QNORM[0]/BOTTHETA
    THETA2 = frac2*QNORM[1]/BOTTHETA

    PHI1 = frac1*RNORM[0]/BOTPHI
    PHI2 = frac2*RNORM[1]/BOTPHI

    L1 = (COORD/2.0)*(RNORM[0]-QNORM[0])-(RNORM[0]-1.0)
    L2 = (COORD/2.0)*(RNORM[1]-QNORM[1])-(RNORM[1]-1.0)


    GAMMASG1 = log(PHI1/frac1)+(COORD/2)*QNORM[0]*log(THETA1/PHI1)+L1-(PHI1/frac1)* (frac1*L1 + frac2*L2)
    GAMMASG2 = log(PHI2/frac2)+(COORD/2)*QNORM[1]*log(THETA2/PHI2)+L2-(PHI2/frac2)* (frac1*L1 + frac2*L2)

    SUMGAMMA1 = 0.0
    SUMGAMMA2 = 0.0
    for i in range(COMPSEG):
        SUMGAMMA1 = SUMGAMMA1 +((profiles[0][i]/AEFFPRIME)*(log(SEGGAMMA[i]/SEGGAMMAPR1[i])))
        SUMGAMMA2 = SUMGAMMA2 +((profiles[1][i]/AEFFPRIME)*(log(SEGGAMMA[i]/SEGGAMMAPR2[i])))




    GAMMA =exp(SUMGAMMA1 + GAMMASG1)
    GAMMA2=exp(SUMGAMMA2 + GAMMASG2)
    LNGAMMA = log(GAMMA)
    LNGAMMA2 = log(GAMMA2)

*

IndexError: list index out of range

In [11]:
%%timeit
while True:
    SEGGAMMAOLD = copy(SEGGAMMA)
    for i in range(COMPSEG):

        SUMMATION=0


        for j in range(COMPSEG):


            SUMMATION = SUMMATION + profile[j]* SEGGAMMAOLD[j] * np.exp(-DELTAW[i,j]/(RGAS*SYSTEMP))

        SEGGAMMA[i]=np.exp(-np.log(SUMMATION))
        SEGGAMMA[i]=(SEGGAMMA[i]+SEGGAMMAOLD[i])/2
    for i in range(COMPSEG):
        CONVERG[i]=abs((SEGGAMMA[i]-SEGGAMMAOLD[i])/SEGGAMMAOLD[i])
    if max(CONVERG)<=0.0000001:
        break


NameError: name 'SEGGAMMA' is not defined

In [12]:
cot

NameError: name 'cot' is not defined